In [ ]:
url = "https://www.iplt20.com/match/2025/1838"

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd


In [ ]:
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
inning_tabs = driver.find_elements(By.CSS_SELECTOR, "a.ap-inner-tb-click")

In [ ]:
for i in inning_tabs:
    print(i.text)

In [ ]:
if len(inning_tabs) < 2:
    raise ValueError("Less than two innings tabs found.")

In [ ]:
first_tab = inning_tabs[0]
first_team = first_tab.text.strip()
first_tab.click()
time.sleep(2)
first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
second_tab = inning_tabs[1]
second_team = second_tab.text.strip()
second_tab.click()
time.sleep(2)
second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
first_inning_soup, second_inning_soup, whos_first_inning, whos_second_inning = first_inning_soup, second_inning_soup, first_team, second_team

In [ ]:
def get_match_metadata(soup):
    """
    The function provides metadata of the match up between two teams.
    Input : Provide the soup of the innings html data.
    Ouput : Gives out a array of [Match Number, Venue, Date, Time].
    """
    l = []
    match_number = soup.find(class_ = "matchOrder mob-hide ng-binding ng-scope").text 
    l.append(match_number)
    
    for i in soup.find(class_ = "ap-match-place col-100 floatLft textCenter re ng-scope").find_all('span'):
        l.append(i.text)
    return l

In [ ]:
match_metadata = get_match_metadata(first_inning_soup)
match_metadata

In [ ]:
def get_ball_by_ball_commentary(soup):
    # ball_wrapper = soup.find_all('div', class_ = ['ballWrapper ng-scope','cmdOver mcBall mcBall mcBall mcBall mcBall mcBall'])
    ball_wrapper = soup.select('div.ballWrapper.ng-scope')
    ball_wrapper = ball_wrapper[1:]
    
    # To Store the scrapped data
    l = []

    for i in ball_wrapper:
        if i.p.contents[0].strip() != '':

            # Which ball of the over is this
            ball_number = i.p.contents[0].strip()

            # Event happened on this ball
            event_on_that_ball = i.find('i').text.strip()

            # bowler vs batter info
            bowling_info_tag = i.find('div', class_='commentaryStartText')
            bowling_info = bowling_info_tag.get_text(strip=True) if bowling_info_tag else None

            # detailed commentary
            commentary_tag = i.find('div', class_='commentaryText')
            commentary = commentary_tag.get_text(strip=True) if commentary_tag else None

            l.append({
                'ball': ball_number,
                'event': event_on_that_ball,
                'bowling_info': bowling_info,
                'commentary': commentary,
            })

    df = pd.DataFrame(l)
    return df

In [ ]:
df1 = get_ball_by_ball_commentary(first_inning_soup)

In [ ]:
df2 = get_ball_by_ball_commentary(first_inning_soup)


In [ ]:
df1

In [ ]:
df2

In [ ]:
def get_match_soups(url):

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    
    driver = webdriver.Chrome(options = options)
    driver.get(url)

    # Use explicit wait instead of sleep
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ap-inner-tb-click")))

    inning_tabs = driver.find_elements(By.CSS_SELECTOR, "a.ap-inner-tb-click")

    if len(inning_tabs) < 2:
        raise ValueError("Less than two innings tabs found.")

    # First innings
    first_tab = inning_tabs[0]
    first_team = first_tab.text.strip()
    first_tab.click()
    time.sleep(2)
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Second innings
    second_tab = inning_tabs[1]
    second_team = second_tab.text.strip()
    second_tab.click()
    time.sleep(2)
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup, second_inning_soup, first_team, second_team

In [ ]:
a,b,c,d = get_match_soups(url)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_match_soups(url):
    options = Options()
    # Commented out headless mode for debugging
    # options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Debug step: Print first 1000 characters of the page source for inspection
    print(driver.page_source[:1000])  # Print first 1000 chars
    driver.save_screenshot("debug.png")  # Save a screenshot for inspection

    # Wait for an earlier element (before tabs) to confirm the page is loaded
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.some-container-or-tab-wrapper")))  # Replace with a more reliable element

    # Now, wait for the innings tabs
    inning_tabs = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ap-inner-tb-click"))
    )

    if len(inning_tabs) < 2:
        raise ValueError("Less than two innings tabs found.")

    # First innings
    first_tab = inning_tabs[0]
    first_team = first_tab.text.strip()
    first_tab.click()
    time.sleep(2)  # You can replace this with another WebDriverWait if needed
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Re-find the inning tabs after page update (to avoid stale elements)
    inning_tabs = driver.find_elements(By.CSS_SELECTOR, "a.ap-inner-tb-click")

    # Second innings
    second_tab = inning_tabs[1]
    second_team = second_tab.text.strip()
    second_tab.click()
    time.sleep(2)  # Again, consider WebDriverWait here
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup, second_inning_soup, first_team, second_team


In [ ]:
a,b,c,d = get_match_soups(url)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_match_soups(url):
    options = Options()
    # Comment out headless mode for debugging
    # options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Debug step: Print first 1000 characters of the page source for inspection
    print(driver.page_source[:1000])  # Print first 1000 chars
    driver.save_screenshot("debug.png")  # Save a screenshot for inspection

    # Wait for a common container element (replace with an actual container)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.some-container-or-tab-wrapper")))  # Replace with a real selector

    # Now, wait for the innings tabs
    inning_tabs = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ap-inner-tb-click"))
    )

    if len(inning_tabs) < 2:
        raise ValueError("Less than two innings tabs found.")

    # First innings
    first_tab = inning_tabs[0]
    first_team = first_tab.text.strip()
    first_tab.click()
    time.sleep(2)  # Replace with WebDriverWait if needed
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Re-find the inning tabs after the first tab click (avoid stale reference)
    inning_tabs = driver.find_elements(By.CSS_SELECTOR, "a.ap-inner-tb-click")

    # Second innings
    second_tab = inning_tabs[1]
    second_team = second_tab.text.strip()
    second_tab.click()
    time.sleep(2)  # Again, consider WebDriverWait here
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup, second_inning_soup, first_team, second_team


In [ ]:
a,b,c,d = get_match_soups(url)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_match_soups(url):
    options = Options()
    # options.add_argument('--headless')  # Uncomment for headless mode
    options.add_argument('--disable-blink-features=AutomationControlled')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Wait until the page has fully loaded (adjust for visible element in the header)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.main-header")))  # Replace with an actual element you know will be on page

    # Debug: Print the first 1000 chars of page source to verify the content
    print(driver.page_source[:1000])
    driver.save_screenshot("debug.png")  # Screenshot for visual confirmation

    # Wait for the innings tabs to be clickable
    inning_tabs = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.ap-inner-tb-click"))
    )

    # Check that we found enough tabs
    if len(inning_tabs) < 2:
        raise ValueError("Less than two innings tabs found.")

    # First innings
    first_tab = inning_tabs[0]
    first_team = first_tab.text.strip()
    first_tab.click()
    time.sleep(2)  # Wait for the content to load

    # Parse the first innings
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Re-fetch the tabs after page change to avoid stale reference
    inning_tabs = driver.find_elements(By.CSS_SELECTOR, "a.ap-inner-tb-click")

    # Second innings
    second_tab = inning_tabs[1]
    second_team = second_tab.text.strip()
    second_tab.click()
    time.sleep(2)  # Wait for the content to load

    # Parse the second innings
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup, second_inning_soup, first_team, second_team


In [ ]:
a,b,c,d = get_match_soups(url)


In [ ]:
import requests

# url = "your_url_here"
response = requests.get(url)

# Print the status code to check if you are being blocked
print(f"HTTP Response Code: {response.status_code}")

if response.status_code == 403:
    print("Access Forbidden - Blocked")
elif response.status_code == 429:
    print("Rate-limited - Too many requests")
elif response.status_code == 200:
    print("Page Loaded Successfully!")
else:
    print("Unexpected response code:", response.status_code)


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_match_soups(url):
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid detection
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")  # Mimic a real browser

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Wait for the innings tabs to appear
    wait = WebDriverWait(driver, 15)
    inning_tabs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ap-inner-tb-click")))

    # Print the number of tabs found for verification
    print(f"Found {len(inning_tabs)} innings tabs.")

    # Continue with the rest of the scraping as usual
    if len(inning_tabs) < 2:
        raise ValueError("Less than two innings tabs found.")
    
    # First innings
    first_tab = inning_tabs[0]
    first_team = first_tab.text.strip()
    first_tab.click()
    time.sleep(2)
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Second innings
    second_tab = inning_tabs[1]
    second_team = second_tab.text.strip()
    second_tab.click()
    time.sleep(2)
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup, second_inning_soup, first_team, second_team

# Run the function
# url = "your_url_here"
first_inning_soup, second_inning_soup, first_team, second_team = get_match_soups(url)
